In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle

# Import Feature Engineered Sales Transaction file
sales_df = pd.read_csv('Sales-Transactions-Edited.csv')


# Items a Customer purchased the most

In [6]:
# Find the no of units sold of each product by customer
top_sell_cust_items_df = sales_df.groupby(['Party','Product']).agg({'Qty':'sum'})

# Reset the index by converting the Party and Product into a column
top_sell_cust_items_df.reset_index(inplace=True)


# Rank the product by most Qty sold, at Customer level
party_col = top_sell_cust_items_df['Party']
qty_col = top_sell_cust_items_df['Qty'].astype(str)
top_sell_cust_items_df['Top_Sell_Rank'] = (party_col + qty_col).rank(method='min',ascending=False).astype(int)


# List the top 20 items sold
top_sell_cust_items_df.sort_values('Top_Sell_Rank',ascending=True).head(20)

,Party,Product,Qty,Top_Sell_Rank
19726,YVT-PMR,VISHNU 250ML,600,1
19722,YVT-PMR,NO-00,5,2
19725,YVT-PMR,TIRUMALA KIRANA,43,3
19724,YVT-PMR,NO-2,10,4
19720,YVT-PMR,N0-5,10,4
19723,YVT-PMR,NO-1,10,4
19721,YVT-PMR,NO-0,10,4
19715,YVSR-VGIRI,NO-2,9,8
19712,YVSR-VGIRI,NO-0,8,9
19711,YVSR-VGIRI,N0-5,5,10


# Items a Customer frequently purchased

In [7]:
# Considered Date column instead of Voucher, in counting the no of orders placed for a product.
# This ignores the multiple no of orders created in a single day.
# Here the understanding is that, this being a wholesale business,
#      a customer places a 2nd order of the same product in a day, only when he/she notices a wrong qty placed on the order.
# If the business considers to have Voucher column, instead of Date column, all the Date column below needs to be replaced.


# Remove duplicate records at Party, Product and Date level
unique_order_items_df = sales_df.drop_duplicates(['Party','Product','Date'])


# Find the no of orders placed and the unique no of customers placed orders, of each product
freq_items_df = unique_order_items_df.groupby(['Party','Product']).agg({'Date':'count'})
freq_items_df.columns=['No_of_Orders']

# Reset the index by converting the Party and Product into columns
freq_items_df.reset_index(inplace=True)


# Products with high no of orders are considered as most frequently purchased items

# Rank the product by No of Orders, at Customer Level
party_col = freq_items_df['Party']
ord_count_col = freq_items_df['No_of_Orders'].astype(str)
freq_items_df['Popularity_Rank'] = (party_col + ord_count_col).rank(method='min',ascending=False).astype(int)


# List of top 20 most popular items sold
freq_items_df.sort_values('Popularity_Rank',ascending=True).head(20)

,Party,Product,No_of_Orders,Popularity_Rank
19726,YVT-PMR,VISHNU 250ML,4,1
19725,YVT-PMR,TIRUMALA KIRANA,1,2
19724,YVT-PMR,NO-2,1,2
19723,YVT-PMR,NO-1,1,2
19722,YVT-PMR,NO-00,1,2
19721,YVT-PMR,NO-0,1,2
19720,YVT-PMR,N0-5,1,2
19715,YVSR-VGIRI,NO-2,2,8
19710,YVSR-VGIRI,GANESH 16*20,2,8
19709,YVSR-VGIRI,GANESH 13*16,2,8


# Merge all the Ranks

In [8]:
# Merge Top Selling Items Rank and Popularity Rank dataframes
cust_prod_rankings_df = pd.merge(top_sell_cust_items_df,freq_items_df,how='inner',on=['Party','Product'])


# Merge the Unit Price (max price at product level)

# Find the unit price of each product (max of price considered, may required to be changed to median or mean)
items_price_df = sales_df.groupby(['Product']).agg({'Rate':'max'})

# Reset the index by converting the Party and Product into columns
items_price_df.reset_index(inplace=True)

# This ensures the same unit price is attached to the product purchased by different customers
cust_prod_rankings_df = pd.merge(cust_prod_rankings_df,items_price_df,how='left',on='Product')


# Get only the Customer, Product, Price and Rank columns
cust_prod_rankings_df = cust_prod_rankings_df[['Party','Product','Rate','Qty','Top_Sell_Rank','No_of_Orders','Popularity_Rank']]


# List the Product Rankings
cust_prod_rankings_df.sort_values('Popularity_Rank',ascending=True).head(20)

,Party,Product,Rate,Qty,Top_Sell_Rank,No_of_Orders,Popularity_Rank
19726,YVT-PMR,VISHNU 250ML,35.0,600,1,4,1
19725,YVT-PMR,TIRUMALA KIRANA,160.0,43,3,1,2
19724,YVT-PMR,NO-2,160.0,10,4,1,2
19723,YVT-PMR,NO-1,160.0,10,4,1,2
19722,YVT-PMR,NO-00,160.0,5,2,1,2
19721,YVT-PMR,NO-0,1540.0,10,4,1,2
19720,YVT-PMR,N0-5,160.0,10,4,1,2
19715,YVSR-VGIRI,NO-2,160.0,9,8,2,8
19710,YVSR-VGIRI,GANESH 16*20,320.0,100,18,2,8
19709,YVSR-VGIRI,GANESH 13*16,260.0,150,16,2,8


# Write the Customer Product Rankings into a .csv file

In [9]:
cust_prod_rankings_df.to_csv('Customer-Product-Rankings.csv',index=False)

# Create a Pickle (.pkl) file with the Ranking dataframe¶

In [10]:
pickle.dump(cust_prod_rankings_df, open('cust_prod_ranking_model.pkl','wb'))